In [15]:
from Utils.Constants import Constants

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

emb_dim: int  = 256
batch_size = 16
vocab_size = 40
seq_len = 20
lstm_hidden = 32
lstm_layer=2 
dropout = .8

fake_data = torch.randint(0, 10, (batch_size, seq_len))

In [4]:
baseline_model = BaselineLSTMModel(
    vocab_size= vocab_size,
    embedding_dim= emb_dim,
    lstm_hidden=lstm_hidden,
    dropout=dropout,
    num_lstm_layers= lstm_layer,
    paddingValue= 0,
)

In [7]:
baseline_model(fake_data).shape

torch.Size([16, 20, 40])

In [11]:
from torch.utils.data import DataLoader
from Data.BPI2012Dataset import BPI2012Dataset
bpi2012 = BPI2012Dataset('../Data/event_logs/BPI_Challenge_2012.xes')
loader = DataLoader(bpi2012, batch_size=32, shuffle=True, collate_fn= bpi2012.collate_fn)
baseline_model = BaselineLSTMModel(
    vocab_size= bpi2012.vocab_size(),
    embedding_dim= 64,
    lstm_hidden= 32,
    dropout= .8,
    num_lstm_layers= 2,
    paddingValue= bpi2012.vocab_to_index(Constants.PAD_VOCAB),
)

parsing log, completed traces :: 100%|██████████| 13087/13087 [00:14<00:00, 900.94it/s]
/Users/jrhs/.pyenv/versions/dl/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [28]:
caseids, train, target, lengths = iter(loader).next()

In [29]:
baseline_model(train, lengths)

RuntimeError: start (845) + length (1) exceeds dimension size (845).

In [33]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class BaselineLSTMModel(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int,  lstm_hidden: int, dropout: float, num_lstm_layers: int, paddingValue: int = 0):
        super(BaselineLSTMModel, self).__init__()

        self.emb = nn.Embedding(vocab_size, embedding_dim, paddingValue)
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden, batch_first=True,
                            dropout=dropout, num_layers=num_lstm_layers)

        self.output_net = nn.Sequential(
            nn.Linear(lstm_hidden, vocab_size),
        )

        # The trainable init h0 and c0 in LSTM.
        self.h0 = nn.Parameter(torch.randn(num_lstm_layers, 1, lstm_hidden))
        self.c0 = nn.Parameter(torch.randn(num_lstm_layers, 1, lstm_hidden))

    def forward(self, input: torch.tensor, lengths: np.ndarray = None) -> torch.tensor:
        '''
        Input size: (B ,S)
        Output size: (B, S, vocab_size)
        '''
        # input (B, S)
        batch_size = input.size(0)
        out = self.emb(input)  # ( B, S, F )
        print("after emb")

        if not lengths is None:
            print("the len of lengths: {}".format(len(lengths)))
            print("the max length: {}", lengths.max())
            print("current tensor size {}". format(out.shape))
            print('in length')
            out = pack_padded_sequence(out, lengths=lengths, batch_first=True)
            print('after pack')
            out, _ = self.lstm(out, (self.h0.repeat(
                1, batch_size, 1), self.c0.repeat(1, batch_size, 1)))  # ( B, S, F)
            print('after lstm')

            out, _ = pad_packed_sequence(out, batch_first=True)
            print('after pad')

        else:
            out, _ = self.lstm(out, (self.h0.repeat(
                1, batch_size, 1), self.c0.repeat(1, batch_size, 1)))  # ( B, S, F)

        out = self.output_net(out)  # (B, S, vocab_size)
        print("after outnet")
        return out

    def argmax_prediction(self, input: torch.tensor, lengths: np.ndarray = None, onlyReturnFinalStep: bool = True) -> torch.tensor:
        seq_size = input.size(1)
        out = self.forward(input)  # (B, S, vocab_size)
        print("after forward")
        out = F.softmax(out, dim=-1)  # (B, S, vocab_size)
        print("after softmax")
        out = torch.argmax(out, dim=-1)  # (B, S)
        if onlyReturnFinalStep:
            final_index = torch.tensor([l - 1 for l in lengths])
            final_out_mask = torch.gt(F.one_hot(final_index, seq_size), 0)
            out =  out.masked_select(final_out_mask) # (B)
        return out

In [2]:
from torch.utils.data import DataLoader
from Data.BPI2012Dataset import BPI2012Dataset
bpi2012 = BPI2012Dataset('../Data/event_logs/BPI_Challenge_2012.xes')
loader = DataLoader(bpi2012, batch_size=32, shuffle=True, collate_fn= bpi2012.collate_fn)


/Users/jrhs/.pyenv/versions/dl/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
parsing log, completed traces :: 100%|██████████| 13087/13087 [00:15<00:00, 834.58it/s]
/Users/jrhs/.pyenv/versions/dl/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [34]:
from Utils.Constants import Constants

baseline_model = BaselineLSTMModel(
    vocab_size= bpi2012.vocab_size(),
    embedding_dim= 64,
    lstm_hidden= 32,
    dropout= .8,
    num_lstm_layers= 2,
    paddingValue= bpi2012.vocab_to_index(Constants.PAD_VOCAB),
)

In [35]:
caseids, train, target, lengths = iter(loader).next()

In [36]:
out = baseline_model(train, lengths)

after emb
the len of lengths: 32
the max length: {} 87
current tensor size torch.Size([32, 87, 64])
in length
after pack
after lstm
after pad
after outnet


In [37]:
out.shape

torch.Size([32, 87, 39])

In [39]:
prediction_out = baseline_model.argmax_prediction(train, lengths)

after emb
after outnet
after forward
after softmax


In [30]:
type(train.size()[0])

int

In [40]:
len(prediction_out)

32

In [41]:
train.shape

torch.Size([32, 87])